In [1]:
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data
from datetime import datetime
#载入数据
mnist = input_data.read_data_sets("/root/code/MNIST_data/",one_hot=True)
#载入图片是 28*28
n_inputs = 28 #输入一共28行
max_time = 28 #一共28行
lstm_size = 100#隐层单元 block
n_classes = 10 #10个分类
batch_size = 50 #每批次50个样本
n_batch = mnist.train.num_examples//batch_size #计算一共有多少批次

#这里的none 表示第一维度可以为任意长度
x = tf.placeholder(tf.float32,[None,784],name='x')
#正确的标签
y = tf.placeholder(tf.float32,[None,10],name='y')

#初始化权重
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
#初始化偏置
biases =tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义run网络
def RUN(X,weights,biases,re_name):
    # inputs = [batch_size,max_time,n_inputs]
    inputs = tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本CELL
    """
    tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True): 
    n_hidden表示神经元的个数，
    forget_bias就是LSTM们的忘记系数，默认为1，如果等于1，就是不会忘记任何信息。如果等于0，就都忘记。
    state_is_tuple默认就是True，官方建议用True，就是表示返回的状态用一个元祖表示。
    这个里面存在一个状态初始化函数，就是zero_state（batch_size，dtype）两个参数。
    batch_size就是输入样本批次的数目，
    dtype就是数据类型。
    """
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    #final_state[0]是cell state
    #final_state[1]是hidden state
    """
    tf.nn.dynamic_rnn(cell,inputs,sequence_length=None,initial_state=None,dtype=None,parallel_iterations=None,swap_memory=False,
    time_major=False,scope=None )
    cell: RNNCell的一个实例.

inputs: RNN输入.

如果time_major == False(默认), 则是一个shape为[batch_size, max_time, input_size]的Tensor,或者这些元素的嵌套元组。

如果time_major == True,则是一个shape为[max_time, batch_size, input_size]的Tensor,或这些元素的嵌套元组。

sequence_length: (可选）大小为[batch_size],数据的类型是int32/int64向量。如果当前时间步的index超过该序列的实际长度时，
则该时间步不进行计算，RNN的state复制上一个时间步的，同时该时间步的输出全部为零。

initial_state: (可选)RNN的初始state(状态)。如果cell.state_size(一层的RNNCell)是一个整数，
那么它必须是一个具有适当类型和形状的张量[batch_size，cell.state_size]。如果cell.state_size是一个元组(多层的RNNCell,如MultiRNNCell)，
那么它应该是一个张量元组，每个元素的形状为[batch_size，s] for s in cell.state_size。

time_major: inputs 和outputs 张量的形状格式。如果为True，则这些张量都应该是（都会是）[max_time, batch_size, depth]。
如果为false，则这些张量都应该是（都会是）[batch_size，max_time, depth]。time_major=true说明输入和输出tensor的第一维是max_time。
否则为batch_size。

使用time_major =True更有效,因为它避免了RNN计算开始和结束时的转置.但是,大多数TensorFlow数据都是batch-major,因此默认情况下,
此函数接受输入并以batch-major形式发出输出.

返回值：

一对(outputs, state),其中：

outputs： RNN输出Tensor.

如果time_major == False(默认),这将是shape为[batch_size, max_time, cell.output_size]的Tensor.

如果time_major == True,这将是shape为[max_time, batch_size, cell.output_size]的Tensor.

state： 最终的状态.

一般情况下state的形状为 [batch_size, cell.output_size ]

如果cell是LSTMCells,则state将是包含每个单元格的LSTMStateTuple的元组，state的形状为[2，batch_size, cell.output_size ]

    """
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases,name=re_name)
    return results

#计算RUN的返回结果
result = "result"
prediction = RUN(x,weights,biases,result)
#损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels =y))
#使用Adamoptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔型列表中
correct_prediction= tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#求准确率

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

saver = tf.train.Saver() #定义saver


#初始化
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    start = datetime.now()
    for epoch in range(5001):
#         for batch in range(n_batch):
        batch_xs,batch_ys =mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter"+str(epoch)+",testing accuracy="+ str(acc))
    end = datetime.now()
    latency = (end - start).total_seconds()
    print('训练5000步总时长为 '+str(latency)+' 秒 ')
    saver.save(sess, '/tmp/tf_rnn_1/model.ckpt') #模型储存位置




Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /root/code/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /root/code/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /root/code/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /root/code/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Pleas

Iter141,testing accuracy=0.2702
Iter142,testing accuracy=0.2703
Iter143,testing accuracy=0.2703
Iter144,testing accuracy=0.2709
Iter145,testing accuracy=0.2719
Iter146,testing accuracy=0.2723
Iter147,testing accuracy=0.2747
Iter148,testing accuracy=0.2763
Iter149,testing accuracy=0.2765
Iter150,testing accuracy=0.2783
Iter151,testing accuracy=0.2797
Iter152,testing accuracy=0.2795
Iter153,testing accuracy=0.2813
Iter154,testing accuracy=0.2833
Iter155,testing accuracy=0.2842
Iter156,testing accuracy=0.2855
Iter157,testing accuracy=0.2856
Iter158,testing accuracy=0.2852
Iter159,testing accuracy=0.2859
Iter160,testing accuracy=0.2868
Iter161,testing accuracy=0.2875
Iter162,testing accuracy=0.2881
Iter163,testing accuracy=0.2883
Iter164,testing accuracy=0.2892
Iter165,testing accuracy=0.2903
Iter166,testing accuracy=0.2914
Iter167,testing accuracy=0.2915
Iter168,testing accuracy=0.2927
Iter169,testing accuracy=0.2948
Iter170,testing accuracy=0.2958
Iter171,testing accuracy=0.2957
Iter172,

Iter398,testing accuracy=0.5124
Iter399,testing accuracy=0.5129
Iter400,testing accuracy=0.5131
Iter401,testing accuracy=0.5127
Iter402,testing accuracy=0.5128
Iter403,testing accuracy=0.5125
Iter404,testing accuracy=0.5133
Iter405,testing accuracy=0.5124
Iter406,testing accuracy=0.5126
Iter407,testing accuracy=0.51
Iter408,testing accuracy=0.5092
Iter409,testing accuracy=0.5052
Iter410,testing accuracy=0.5018
Iter411,testing accuracy=0.5002
Iter412,testing accuracy=0.4995
Iter413,testing accuracy=0.4997
Iter414,testing accuracy=0.5022
Iter415,testing accuracy=0.5053
Iter416,testing accuracy=0.5095
Iter417,testing accuracy=0.5113
Iter418,testing accuracy=0.5121
Iter419,testing accuracy=0.5116
Iter420,testing accuracy=0.5099
Iter421,testing accuracy=0.5074
Iter422,testing accuracy=0.5069
Iter423,testing accuracy=0.5063
Iter424,testing accuracy=0.5061
Iter425,testing accuracy=0.5066
Iter426,testing accuracy=0.5087
Iter427,testing accuracy=0.5115
Iter428,testing accuracy=0.5127
Iter429,te

Iter655,testing accuracy=0.6313
Iter656,testing accuracy=0.6367
Iter657,testing accuracy=0.6398
Iter658,testing accuracy=0.6403
Iter659,testing accuracy=0.6417
Iter660,testing accuracy=0.6425
Iter661,testing accuracy=0.6439
Iter662,testing accuracy=0.6442
Iter663,testing accuracy=0.6439
Iter664,testing accuracy=0.644
Iter665,testing accuracy=0.6448
Iter666,testing accuracy=0.6467
Iter667,testing accuracy=0.6461
Iter668,testing accuracy=0.6456
Iter669,testing accuracy=0.6452
Iter670,testing accuracy=0.6424
Iter671,testing accuracy=0.6399
Iter672,testing accuracy=0.6395
Iter673,testing accuracy=0.6382
Iter674,testing accuracy=0.637
Iter675,testing accuracy=0.6352
Iter676,testing accuracy=0.6392
Iter677,testing accuracy=0.6409
Iter678,testing accuracy=0.6424
Iter679,testing accuracy=0.6452
Iter680,testing accuracy=0.647
Iter681,testing accuracy=0.6466
Iter682,testing accuracy=0.6466
Iter683,testing accuracy=0.6466
Iter684,testing accuracy=0.648
Iter685,testing accuracy=0.649
Iter686,testi

Iter912,testing accuracy=0.706
Iter913,testing accuracy=0.7069
Iter914,testing accuracy=0.7043
Iter915,testing accuracy=0.7031
Iter916,testing accuracy=0.7021
Iter917,testing accuracy=0.7018
Iter918,testing accuracy=0.7013
Iter919,testing accuracy=0.7008
Iter920,testing accuracy=0.7012
Iter921,testing accuracy=0.7005
Iter922,testing accuracy=0.7011
Iter923,testing accuracy=0.7015
Iter924,testing accuracy=0.7024
Iter925,testing accuracy=0.7016
Iter926,testing accuracy=0.7006
Iter927,testing accuracy=0.6998
Iter928,testing accuracy=0.6984
Iter929,testing accuracy=0.6996
Iter930,testing accuracy=0.6991
Iter931,testing accuracy=0.6994
Iter932,testing accuracy=0.6999
Iter933,testing accuracy=0.7022
Iter934,testing accuracy=0.7061
Iter935,testing accuracy=0.7081
Iter936,testing accuracy=0.7092
Iter937,testing accuracy=0.709
Iter938,testing accuracy=0.71
Iter939,testing accuracy=0.7096
Iter940,testing accuracy=0.7122
Iter941,testing accuracy=0.7134
Iter942,testing accuracy=0.7139
Iter943,test

...

Iter3911,testing accuracy=0.9036
Iter3912,testing accuracy=0.9039
Iter3913,testing accuracy=0.9054
Iter3914,testing accuracy=0.9075
Iter3915,testing accuracy=0.908
Iter3916,testing accuracy=0.9091
Iter3917,testing accuracy=0.91
Iter3918,testing accuracy=0.9103
Iter3919,testing accuracy=0.9109
Iter3920,testing accuracy=0.9107
Iter3921,testing accuracy=0.9104
Iter3922,testing accuracy=0.9109
Iter3923,testing accuracy=0.9115
Iter3924,testing accuracy=0.9128
Iter3925,testing accuracy=0.9129
Iter3926,testing accuracy=0.9131
Iter3927,testing accuracy=0.9131
Iter3928,testing accuracy=0.9129
Iter3929,testing accuracy=0.9132
Iter3930,testing accuracy=0.9129
Iter3931,testing accuracy=0.9118
Iter3932,testing accuracy=0.911
Iter3933,testing accuracy=0.9103
Iter3934,testing accuracy=0.9088
Iter3935,testing accuracy=0.9099
Iter3936,testing accuracy=0.9092
Iter3937,testing accuracy=0.9073
Iter3938,testing accuracy=0.9062
Iter3939,testing accuracy=0.904
Iter3940,testing accuracy=0.9016
Iter3941,testin

Iter4161,testing accuracy=0.911
Iter4162,testing accuracy=0.9121
Iter4163,testing accuracy=0.9127
Iter4164,testing accuracy=0.9133
Iter4165,testing accuracy=0.9134
Iter4166,testing accuracy=0.9131
Iter4167,testing accuracy=0.9129
Iter4168,testing accuracy=0.9125
Iter4169,testing accuracy=0.9123
Iter4170,testing accuracy=0.9119
Iter4171,testing accuracy=0.9123
Iter4172,testing accuracy=0.9117
Iter4173,testing accuracy=0.9112
Iter4174,testing accuracy=0.9105
Iter4175,testing accuracy=0.9102
Iter4176,testing accuracy=0.9101
Iter4177,testing accuracy=0.9104
Iter4178,testing accuracy=0.911
Iter4179,testing accuracy=0.9118
Iter4180,testing accuracy=0.9114
Iter4181,testing accuracy=0.9114
Iter4182,testing accuracy=0.9114
Iter4183,testing accuracy=0.911
Iter4184,testing accuracy=0.9115
Iter4185,testing accuracy=0.9113
Iter4186,testing accuracy=0.9098
Iter4187,testing accuracy=0.909
Iter4188,testing accuracy=0.9082
Iter4189,testing accuracy=0.9084
Iter4190,testing accuracy=0.9093
Iter4191,testi

Iter4411,testing accuracy=0.9114
Iter4412,testing accuracy=0.9121
Iter4413,testing accuracy=0.9122
Iter4414,testing accuracy=0.9132
Iter4415,testing accuracy=0.9134
Iter4416,testing accuracy=0.913
Iter4417,testing accuracy=0.9131
Iter4418,testing accuracy=0.9124
Iter4419,testing accuracy=0.9126
Iter4420,testing accuracy=0.9132
Iter4421,testing accuracy=0.9132
Iter4422,testing accuracy=0.9133
Iter4423,testing accuracy=0.9121
Iter4424,testing accuracy=0.9114
Iter4425,testing accuracy=0.9113
Iter4426,testing accuracy=0.9133
Iter4427,testing accuracy=0.9147
Iter4428,testing accuracy=0.9166
Iter4429,testing accuracy=0.9172
Iter4430,testing accuracy=0.9176
Iter4431,testing accuracy=0.9168
Iter4432,testing accuracy=0.9158
Iter4433,testing accuracy=0.9138
Iter4434,testing accuracy=0.912
Iter4435,testing accuracy=0.9126
Iter4436,testing accuracy=0.911
Iter4437,testing accuracy=0.9132
Iter4438,testing accuracy=0.9131
Iter4439,testing accuracy=0.9129
Iter4440,testing accuracy=0.9126
Iter4441,test

Iter4660,testing accuracy=0.9172
Iter4661,testing accuracy=0.9175
Iter4662,testing accuracy=0.917
Iter4663,testing accuracy=0.9158
Iter4664,testing accuracy=0.9154
Iter4665,testing accuracy=0.9144
Iter4666,testing accuracy=0.9142
Iter4667,testing accuracy=0.914
Iter4668,testing accuracy=0.9149
Iter4669,testing accuracy=0.9165
Iter4670,testing accuracy=0.9175
Iter4671,testing accuracy=0.9183
Iter4672,testing accuracy=0.9179
Iter4673,testing accuracy=0.9184
Iter4674,testing accuracy=0.9172
Iter4675,testing accuracy=0.9166
Iter4676,testing accuracy=0.9166
Iter4677,testing accuracy=0.9161
Iter4678,testing accuracy=0.9153
Iter4679,testing accuracy=0.9157
Iter4680,testing accuracy=0.9173
Iter4681,testing accuracy=0.9186
Iter4682,testing accuracy=0.918
Iter4683,testing accuracy=0.9189
Iter4684,testing accuracy=0.9187
Iter4685,testing accuracy=0.9182
Iter4686,testing accuracy=0.9173
Iter4687,testing accuracy=0.9162
Iter4688,testing accuracy=0.9154
Iter4689,testing accuracy=0.9152
Iter4690,test

Iter4910,testing accuracy=0.9241
Iter4911,testing accuracy=0.9244
Iter4912,testing accuracy=0.9246
Iter4913,testing accuracy=0.9246
Iter4914,testing accuracy=0.9232
Iter4915,testing accuracy=0.9223
Iter4916,testing accuracy=0.9199
Iter4917,testing accuracy=0.917
Iter4918,testing accuracy=0.9148
Iter4919,testing accuracy=0.9128
Iter4920,testing accuracy=0.9118
Iter4921,testing accuracy=0.9141
Iter4922,testing accuracy=0.9154
Iter4923,testing accuracy=0.9165
Iter4924,testing accuracy=0.9168
Iter4925,testing accuracy=0.9175
Iter4926,testing accuracy=0.9178
Iter4927,testing accuracy=0.9183
Iter4928,testing accuracy=0.9177
Iter4929,testing accuracy=0.9174
Iter4930,testing accuracy=0.9182
Iter4931,testing accuracy=0.9187
Iter4932,testing accuracy=0.9181
Iter4933,testing accuracy=0.9173
Iter4934,testing accuracy=0.9165
Iter4935,testing accuracy=0.9158
Iter4936,testing accuracy=0.9156
Iter4937,testing accuracy=0.917
Iter4938,testing accuracy=0.9178
Iter4939,testing accuracy=0.917
Iter4940,test

此处说明for循环嵌套 sess训练了 更多的次数 可以去除 训练效果是一样的